### Checkout my blog on Medium around this notebook for detailed Fine-Tuning Method

In [1]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!nvidia-smi -L

In [3]:
import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-9b-it",
    load_in_8bit=True,
    device_map='auto',
)

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-9b-it")


config.json:   0%|          | 0.00/857 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/39.1k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [ ]:
input_text = "Write me a poem about Machine Learning."
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids, max_new_tokens=320)
print(tokenizer.decode(outputs[0]))

## Freezing original weights

In [4]:
for param in model.parameters():
  print(param)
  break

Parameter containing:
tensor([[-0.0510,  0.0106, -0.0422,  ..., -0.0294, -0.0352,  0.0127],
        [-0.0007,  0.0127,  0.0083,  ...,  0.0049, -0.0027, -0.0275],
        [-0.0112,  0.0026,  0.0081,  ..., -0.0006,  0.0043,  0.0033],
        ...,
        [-0.0264,  0.0157,  0.0041,  ..., -0.0588, -0.0361, -0.0080],
        [-0.0522,  0.0139, -0.0004,  ..., -0.0061, -0.0491, -0.0109],
        [-0.0413,  0.0264, -0.0249,  ...,  0.0015, -0.0315, -0.0033]],
       device='cuda:0', dtype=torch.float16, requires_grad=True)


In [4]:
for param in model.parameters():
  param.requires_grad = False
  if param.ndim == 1:
    param.data = param.data.to(torch.float32)

  model.gradient_checkpointing_enable()
  model.enable_input_require_grads()

  class CastOutputToFloat(nn.Sequential):
    def forward(self, x) :
      return super().forward(x).to(torch.float32)

  model.lm_head = CastOutputToFloat(model.lm_head)

## Setting up the LoRa Adapters

In [5]:
def print_trainable_parameters(model):
    """
  printing the number of trainable paramters in the model
  """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}")

In [7]:
for name, module in model.named_modules():
    if 'attn' in name or 'attention' in name:  # Common attention module names
        print(name)
        for sub_name, sub_module in module.named_modules():  # Check sub-modules within attention
            print(f"  - {sub_name}")


model.layers.0.self_attn
  - 
  - q_proj
  - k_proj
  - v_proj
  - o_proj
  - rotary_emb
model.layers.0.self_attn.q_proj
  - 
model.layers.0.self_attn.k_proj
  - 
model.layers.0.self_attn.v_proj
  - 
model.layers.0.self_attn.o_proj
  - 
model.layers.0.self_attn.rotary_emb
  - 
model.layers.0.post_attention_layernorm
  - 
model.layers.1.self_attn
  - 
  - q_proj
  - k_proj
  - v_proj
  - o_proj
  - rotary_emb
model.layers.1.self_attn.q_proj
  - 
model.layers.1.self_attn.k_proj
  - 
model.layers.1.self_attn.v_proj
  - 
model.layers.1.self_attn.o_proj
  - 
model.layers.1.self_attn.rotary_emb
  - 
model.layers.1.post_attention_layernorm
  - 
model.layers.2.self_attn
  - 
  - q_proj
  - k_proj
  - v_proj
  - o_proj
  - rotary_emb
model.layers.2.self_attn.q_proj
  - 
model.layers.2.self_attn.k_proj
  - 
model.layers.2.self_attn.v_proj
  - 
model.layers.2.self_attn.o_proj
  - 
model.layers.2.self_attn.rotary_emb
  - 
model.layers.2.post_attention_layernorm
  - 
model.layers.3.self_attn
  - 
 

In [6]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r = 16,
    lora_alpha = 32,
    target_modules = ["k_proj","v_proj"],
    lora_dropout = 0.05,
    bias = "none",
    task_type = "CAUSAL_LM"
)

lora_model = get_peft_model(model, config)
print_trainable_parameters(model)
print_trainable_parameters(lora_model)

trainable params: 7569408 || all params: 9249275392 || trainable%: 0.08183784868755263
trainable params: 7569408 || all params: 9249275392 || trainable%: 0.08183784868755263


## Training data import

In [ ]:
import transformers
from datasets import load_dataset
data = load_dataset("Abirate/english_quotes")
# data = data.map(lambda samples: tokenizer(samples['quote']), batched=True)

In [ ]:
data

In [ ]:
def merge_columns(example):
  example['prediction'] = example["quote"] + " ->: " + str(example['tags'])
  return example

data['train'] = data['train'].map(merge_columns)
data['train']['prediction'][0]

In [ ]:
 data['train'] = data['train']

In [ ]:
data

In [ ]:
data = data.map(lambda samples: tokenizer(samples['prediction']), batched=True)

In [ ]:
data

## Training

In [ ]:
trainer = transformers.Trainer(
    model = lora_model,
    train_dataset = data['train'],
    args = transformers.TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 1,
        warmup_steps = 10,
        max_steps = 50,
        learning_rate = 2e-4,
        fp16 = False,
        logging_steps = 1,
        output_dir = 'outputs'
    ),
    data_collator = transformers.DataCollatorForLanguageModeling(tokenizer, mlm = False)
)
model.config.use_cache = False
trainer.train()

## Sharing adapter on hub

In [ ]:
lora_model.push_to_hub("philomath-1209/gemma-2-9b-it-tagger-test-adapter",
                      use_auth_token=True,
                      commit_message = "Testing Lora Training method",
                      private=False)

## Loading Adapter from the hub

In [1]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "philomath-1209/gemma-2-9b-it-tagger-test-adapter"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path,
                                            return_dict = True,
                                            load_in_8bit = True,
                                            device_map = 'auto')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

model = PeftModel.from_pretrained(model,peft_model_id)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/30.3M [00:00<?, ?B/s]

In [3]:
batch = tokenizer("“If you Want Best AI Engineer, then hire me” ->: ", return_tensors='pt')

with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, max_new_tokens=50)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))

/tmp/ipykernel_1032/2102834301.py:3: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:2091: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(




 “If you Want Best AI Engineer, then hire me” ->: 
['ai', 'engineer', 'hire', 'me', 'programmer']
['ai', 'engineer', 'hire', 'me', 'programmer'] ->: ['ai', 'engineer', 'hire', 'me', 'programmer']

